Import Library

In [1]:
import os
import music21 as m21
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import importlib as imp
from torch.utils.data import DataLoader
import tensorflow as tf

2024-03-16 14:19:29.610275: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
import a02_transformer
imp.reload(a02_transformer)
import a00_funs_make_symbol_seqs as fmseq
from a01_melody_preprocessor import MelodyPreprocessor
from a02_transformer import TransformerModel
from a04_melody_generator import MelodyGenerator
import a03_train


Using device: cpu


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cpu


In [9]:
## Parameters for Data Preprocessing
time_signature = '4/4'
beats_per_measure=4
step_duration = 0.25  # 0.25 = a 1/16 note 
acceptable_durations = np.arange(0.25, 8.1, 0.25) 


## Import Data and Prepare batches
songs = m21.corpus.search('bach', fileExtensions='xml')
melodies = fmseq.make_melody_symbol_sequences(songs, time_signature, 
                                              acceptable_durations)
preprocessor = MelodyPreprocessor(melodies)
training_dataset = preprocessor.create_training_dataset()
training_batches = DataLoader(training_dataset, shuffle=True,
                              batch_size=128)

print(preprocessor.vocab_size)
print(preprocessor.data_size)
print(preprocessor.seq_length)

/Users/ranli/Documents/python_ve/MS_Pytorch_Thesis/lib/python3.10/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


142
18523
187


In [22]:
def key_padding_mask(seq, pad_token=0):
    return (seq == pad_token)

def look_ahead_mask(dim):
    return nn.Transformer.generate_square_subsequent_mask(dim)

In [25]:
def position_encoding(num_pos, d_model):
    position = torch.arange(num_pos).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                         (-torch.log(torch.tensor(10000.0)) / d_model))
    angles = position * div_term
    pos_encoding = torch.zeros(num_pos, d_model)
    pos_encoding[:, 0::2] = torch.sin(angles)
    pos_encoding[:, 1::2] = torch.cos(angles)
    return pos_encoding.unsqueeze(0)  # Add batch dimension

In [23]:
class TransformerModel(nn.Module):

    def __init__(self, d_model, nhead, dropout, dim_feedforward, vocab_size_padding,
                 num_encoder_layers, num_decoder_layers, device):
        super(TransformerModel, self).__init__()  
        self.d_model = d_model
        self.device = device
        self.embedding = nn.Embedding(vocab_size_padding, d_model).to(device)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, 
                                                        dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_encoder_layers)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, 
                                                        dropout=dropout, batch_first=True)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_decoder_layers)
        self.dropout = nn.Dropout(dropout)
        self.final_layer = nn.Linear(d_model, vocab_size_padding)



    def forward(self, src, tgt):
        src_padding_mask = key_padding_mask(src).to(self.device)
        tgt_padding_mask = key_padding_mask(tgt).to(self.device)
        tgt_mask = look_ahead_mask(tgt.size(-1)).to(self.device)  
        scale_factor = torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32, device=self.device))

        x = self.embedding(src) 
        x *= scale_factor
        x += position_encoding(src.size(-1), self.d_model).to(self.device)
        x = self.dropout(x)
        enc_output = self.encoder(x, src_key_padding_mask=src_padding_mask)

        y = self.embedding(tgt)
        y *= scale_factor
        y += position_encoding(tgt.size(-1), self.d_model).to(self.device)
        y = self.dropout(y)
        dec_output = self.decoder(y, enc_output, tgt_mask=tgt_mask,
                                  tgt_key_padding_mask=tgt_padding_mask)
        output = self.final_layer(dec_output)
        return output

In [18]:
# Model Specification and Training
vocab_size_padding = preprocessor.vocab_size + 1
model = TransformerModel(d_model=128, nhead=2, dim_feedforward=128, dropout=0.1, 
                         vocab_size_padding=vocab_size_padding, 
                         num_encoder_layers=6, num_decoder_layers=6, device=device)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

/Users/ranli/Documents/python_ve/MS_Pytorch_Thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
# epochs = 200
# save_interval = 20
epochs = 5
save_interval =1
save_dir = 'generated'
start_sequence = ['C4-1.0', 'G4-1.0', 'E4-1.0', 'C4-1.0']

for epoch in range(epochs):
    average_loss = a03_train.train_each_step(training_batches, model, 
                                             criterion, optimizer, device)
    print(f'Epoch {epoch + 1}/{epochs}, Average Loss: {average_loss}')

    if epoch > 0 and (epoch + 1) % save_interval == 0:
        melody_generator = MelodyGenerator(model, preprocessor.tokenizer, device)
        new_melody = melody_generator.generate(start_sequence, preprocessor.tokenizer)
        np.savetxt(f"{save_dir}/{epoch + 1}.txt", new_melody, fmt='%s')

/Users/ranli/Documents/python_ve/MS_Pytorch_Thesis/lib/python3.10/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


KeyboardInterrupt: 